# Ver campañas e interacciones

En el primer cuaderno `Personalize_BuildCampaign.ipynb`, creó e implementó de forma correcta un modelo de recomendación por medio del aprendizaje profundo con Amazon Personalize.

Este cuaderno ampliará la información anterior y lo guiará en el proceso de agregar la capacidad de reaccionar al comportamiento de los usuarios en tiempo real. Si la intención del usuario cambia mientras navega en busca de una película, verá recomendaciones actualizadas que se basarán en ese comportamiento.

Además, presentará un código de muestra para simular el comportamiento de los usuarios al momento de seleccionar películas antes de que se proporcionen las recomendaciones.

A continuación, comenzaremos con la importación de las bibliotecas que necesitamos para interactuar con Personalize.

In [ ]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

La línea que se muestra debajo recuperará las variables compartidas del primer cuaderno.

In [ ]:
%store -r

In [ ]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Creación de un rastreador de eventos

Antes de que su sistema de recomendación pueda responder a eventos en tiempo real, necesitará un rastreador de eventos. El código que se muestra a continuación generará uno y se podrá utilizar para continuar con este laboratorio. Puede colocarle el nombre que desee.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## Configuración de los datos de origen

Más arriba, puede ver su ID de seguimiento. Este ID se asignó a una variable, por lo que no se requieren más acciones de su parte. Las líneas que aparecen abajo permitirán configurar los datos utilizados para las recomendaciones, de forma que pueda renderizar la lista de películas más adelante.

In [ ]:
# First load items into memory
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.loc[movie_id]['TITLE']

## Obtención de recomendaciones

Primero, renderizaremos nuevamente las recomendaciones del cuaderno anterior:

In [ ]:
recommendations_df

## Simulación del comportamiento del usuario

Las líneas que se muestran a continuación proporcionan un código de ejemplo que simula la interacción de un usuario con un elemento en particular. Por lo tanto, recibirá recomendaciones distintas a las que tenía en el comienzo.

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

Abajo, esta línea actualizará el rastreador como si el usuario hubiese seleccionado un título en particular.


Si la tabla generada por las celdas que se muestran a continuación no modifica las recomendaciones, intente simplemente con otro número aleatorio de 3 dígitos en la celda de arriba y ejecute ambas celdas de nuevo. Verá una tercera columna, la cual se habrá generado a partir de recomendaciones.

In [ ]:
# Pick a movie, we will use ID 270 or Gattaca
movie_to_click = 270
movie_title_clicked = get_movie_title(movie_to_click)
send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)

Después de ejecutar este bloque, verá los cambios en las recomendaciones, ya que habilitó el rastreador de eventos y envió los eventos al servicio.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

## Conclusión

Ahora puede ver que las recomendaciones se alteran cuando cambia la película con la que interactúa el usuario. Este sistema puede modificarse para cualquier aplicación en la que los usuarios interactúen con una recopilación de elementos. Estas herramientas están disponibles en cualquier momento para explorar las posibilidades con los datos que posee.

Ejecute la celda que se muestra abajo para almacenar los valores que se necesitan para limpiar el cuaderno.

Por último, cuando esté listo para quitar los elementos de su cuenta, abra el cuaderno `Cleanup.ipynb` y ejecute los pasos allí.


In [ ]:
%store event_tracker_arn